<a href="https://colab.research.google.com/github/gunabalan-0411/Machine-Learning/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST Dataset

In [ ]:
from sklearn.datasets import fetch_openml
# real_life, toy, synthetic datasets
# fetch_*, load_*, make_*

mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist.data, mnist.target

print(X.shape)



In [ ]:
# printing the image
import matplotlib.pyplot as plt

def show_img(image_2d):
  matrix = image_2d.reshape(28,28)
  plt.imshow(matrix, cmap='binary')

print("It is",y[7])
show_img(X[7])

In [ ]:
# Simple classification
from sklearn.linear_model import  SGDClassifier

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
y_train_5 = (y_train == '5')
y_test_5 = (y_test == '5')
model = SGDClassifier()
model.fit(X_train, y_train_5)



In [ ]:
print("Is 5?",model.predict([X[0]]))
print(f"actual: {y[0]}")

In [ ]:
from sklearn.model_selection import cross_val_score

cvs = cross_val_score(model, X_train, y_train_5, cv=3, scoring='accuracy')
print(cvs)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier()
dummy_model.fit(X_train, y_train_5)
print(any(dummy_model.predict(X_train)))

cvs = cross_val_score(dummy_model, X_train, y_train_5, cv=3, scoring='accuracy')
print(cvs)